In [20]:
import pandas as pd
import numpy as np
import os
import re
import string
import nltk
import matplotlib.pyplot as plt
plt.rc('figure',figsize=(17,13))
import seaborn as sns
sns.set_style('darkgrid')
import plotly.express as ex
from plotly.subplots import make_subplots
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from wordcloud import WordCloud,STOPWORDS, ImageColorGenerator
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [14]:
os.getcwd()

'/Users/clinton/Desktop/HU/CISC_525/Project/CISC-525-Final-Project'

In [3]:
df = pd.read_csv("covid19_tweets.csv")
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False


In [15]:
df['clean_text'] = df['text'].str.lower()
df['clean_text'] = df['clean_text'].apply(lambda x:re.sub(r"http\S+", "", x))

In [21]:
# Punctuation Removal
punctuation_removal = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', punctuation_removal))
df["clean_text"] = df["clean_text"].apply(lambda text: remove_punctuation(text))
df["clean_text"]

0         if i smelled the scent of hand sanitizers toda...
1         hey yankees yankeespr and mlb  wouldnt it have...
2         diane3443 wdunlap realdonaldtrump trump never ...
3         brookbanktv the one gift covid19 has give me i...
4         25 july  media bulletin on novel coronavirusup...
                                ...                        
179103    thanks iamohmai for nominating me for the who ...
179104               2020 the year of insanity lol covid19 
179105    ctvnews a powerful painting by juan lucena its...
179106    more than 1200 students test positive for covi...
179107    i stop when i see a stop\n\nsabcnews\nizindaba...
Name: clean_text, Length: 179108, dtype: object

In [22]:
# Single character and double space removal
df["clean_text"] = df["clean_text"].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))
df["clean_text"] = df["clean_text"].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
df["clean_text"]

0         ifsmelled the scent of hand sanitizers today o...
1         hey yankees yankeespr and mlb wouldnt it have ...
2         diane3443 wdunlap realdonaldtrump trump never ...
3         brookbanktv the one gift covid19 has give me i...
4         25 july media bulletin on novel coronavirusupd...
                                ...                        
179103    thanks iamohmai for nominating me for the who ...
179104               2020 the year of insanity lol covid19 
179105    ctvnewspowerful painting by juan lucena itstri...
179106    more than 1200 students test positive for covi...
179107    i stop whenseestop sabcnews izindabanews24 dai...
Name: clean_text, Length: 179108, dtype: object

In [23]:
# Stopword Removal
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["clean_text"] = df["clean_text"].apply(lambda text: remove_stopwords(text))
df["clean_text"]

0         ifsmelled scent hand sanitizers today someone ...
1         hey yankees yankeespr mlb wouldnt made sense p...
2         diane3443 wdunlap realdonaldtrump trump never ...
3         brookbanktv one gift covid19 give appreciation...
4         25 july media bulletin novel coronavirusupdate...
                                ...                        
179103    thanks iamohmai nominating wearamask challenge...
179104                       2020 year insanity lol covid19
179105    ctvnewspowerful painting juan lucena itstribut...
179106    1200 students test positive covid19 major univ...
179107    stop whenseestop sabcnews izindabanews24 daily...
Name: clean_text, Length: 179108, dtype: object

In [24]:
# Emoji Removal
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [25]:
df["clean_text"] = df["clean_text"].apply(str)
df["clean_text"] = df["clean_text"].apply(remove_emoji)
df["clean_text"]

0         ifsmelled scent hand sanitizers today someone ...
1         hey yankees yankeespr mlb wouldnt made sense p...
2         diane3443 wdunlap realdonaldtrump trump never ...
3         brookbanktv one gift covid19 give appreciation...
4         25 july media bulletin novel coronavirusupdate...
                                ...                        
179103    thanks iamohmai nominating wearamask challenge...
179104                       2020 year insanity lol covid19
179105    ctvnewspowerful painting juan lucena itstribut...
179106    1200 students test positive covid19 major univ...
179107    stop whenseestop sabcnews izindabanews24 daily...
Name: clean_text, Length: 179108, dtype: object

In [26]:
# Most common words
from collections import Counter
cnt = Counter()
for text in df["clean_text"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('covid19', 104784),
 ('cases', 18545),
 ('coronavirus', 14057),
 ('new', 13793),
 ('amp', 11707),
 ('people', 8985),
 ('deaths', 7396),
 ('pandemic', 7360),
 ('us', 7149),
 ('health', 5123)]

In [27]:
# Sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sid = SIA()
df['sentiments'] = df["clean_text"].apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+',str(x).lower()))))
#print(sentiments)
df['Positive Sentiment'] = df['sentiments'].apply(lambda x: x['pos']+1*(10**-6)) 
df['Neutral Sentiment'] = df['sentiments'].apply(lambda x: x['neu']+1*(10**-6))
df['Negative Sentiment'] = df['sentiments'].apply(lambda x: x['neg']+1*(10**-6))
df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,clean_text,sentiments,Positive Sentiment,Neutral Sentiment,Negative Sentiment
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False,ifsmelled scent hand sanitizers today someone ...,"{'neg': 0.0, 'neu': 0.738, 'pos': 0.262, 'comp...",0.262001,0.738001,0.000001
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False,hey yankees yankeespr mlb wouldnt made sense p...,"{'neg': 0.12, 'neu': 0.684, 'pos': 0.197, 'com...",0.197001,0.684001,0.120001
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False,diane3443 wdunlap realdonaldtrump trump never ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000001,1.000001,0.000001
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False,brookbanktv one gift covid19 give appreciation...,"{'neg': 0.0, 'neu': 0.592, 'pos': 0.408, 'comp...",0.408001,0.592001,0.000001
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False,25 july media bulletin novel coronavirusupdate...,"{'neg': 0.0, 'neu': 0.813, 'pos': 0.187, 'comp...",0.187001,0.813001,0.000001
